# Creating a network of AdEx neurons connected with synapses

In this notebook we first investigate parameter space of the AdEx neuron models, then select candidates for future pyramidal and interneuron parameter sets.

Next we create a network consisting of PCs and interneurons, which we connect by synapses with plasticity mechanisms. 

In [ ]:
%pylab inline

In [ ]:
style.use(('ggplot','seaborn-muted'))

In [ ]:
import pandas as pd

In [ ]:
from brian2 import *

In [ ]:
import input_factory as inpf

In [ ]:
def beautify_spikes(statemon,spikemon,neuron_id):
    vm = statemon[neuron_id].v[:]
    offset = statemon.t[0]#/defaultclock.dt
    spike_times = spikemon.t[spikemon.i == neuron_id]
    for t in spike_times:
        i = int((t-offset) / defaultclock.dt)
        vm[i] = 20*mV
    return vm

## AdEx neuron

In [ ]:
## Parameters that are shared by all neurons

# Neurons
Vth = -50*mV   # rheobase threshold
El = -70*mV     # resting membrane potential
Vcut = -20*mV    # spike detection threshold
deltaT = 2*mV  # spike initiation sharpness
Rin = 500*Mohm  # input resistance of a neuron at rest
gl = 1/Rin



# Synapses
E_e = 0*mV     # Excitatory synaptic reversal potential (AMPA and NMDA receptors)
E_i = -80*mV   # Inhibitory synaptic reversal potential (GABAA receptors)
tau_e = 5*ms   # time scale of excitatory synaptic conductance
tau_i = 10*ms  # time scale of excitatory synaptic conductance



AdEx_equations = Equations('''
dv/dt = (-(v-El) + deltaT*exp((v-Vth)/deltaT) + Rin*(Isyn + Ibias + Iapp - w))/tau_m : volt 
dw/dt = (a*(v-El) - w)/tau_w: amp  # adaptation variable
a : siemens
b : amp
tau_m : second
tau_w : second
Vreset: volt
stim_amp : 1
Ibias : amp
Iapp = stim_amp*input_current(t,i): amp
''')

# Synaptic input
synaptic_equations = Equations("""
Isyn =  - g_e*(v-E_e) - g_i*(v-E_i) : amp
dg_e/dt = -g_e/tau_e : siemens
dg_i/dt = -g_i/tau_i : siemens
""")

In [ ]:
adex_params = pd.read_csv('adex_params.csv',index_col='type')
adex_params

In [ ]:
def convert_table_cell(col_name):
    units = col_name.split(' ')[1][1:-1]

def convert_from_table(row):
    return dict(
        a = float(row['a [nS]'])*nS,
        b = float(row['b [pA]'])*pA,
        tau_w = float(row['tau_u [ms]'])*ms,
        tau_m = float(row['tau_m [ms]'])*ms,
        Vreset = float(row['Vreset [mV]'])*mV,
        C = float(row['tau_m [ms]'])*ms*gl,
        Ibias = 0*pA
    )

In [ ]:
tonic_pars = convert_from_table(adex_params.loc['tonic'])
adapting_pars = convert_from_table(adex_params.loc['adapting'])
bursting_pars = convert_from_table(adex_params.loc['bursting'])
initial_burst_pars = convert_from_table(adex_params.loc['init. burst'])
irregular_pars = convert_from_table(adex_params.loc['irregular'])
transient_pars = convert_from_table(adex_params.loc['transient'])
delayed_pars = convert_from_table(adex_params.loc['delayed'])

### Selecting exemplars of interneurons and PCs by Monte-Carlo

Here we create a group of 10^4 neurons with random parameter values. Next we select only neurons with admissible behavior. 

In [ ]:
start_scope()

Nneurons = 10000

defaultclock.dt = 0.1*ms

G = NeuronGroup(Nneurons, AdEx_equations+synaptic_equations,threshold='v>Vcut', reset='v=Vreset; w += b',
                method='exponential_euler')

G.set_states(dict(v=El,w=0))

M = StateMonitor(G, ['v','w'], record=True)
S = SpikeMonitor(G,)

In [ ]:
G.a = uniform(-1,3,Nneurons)*nsiemens
G.b = uniform(5,50,Nneurons)*pA
G.tau_m = uniform(5,200,Nneurons)*ms
G.tau_w = uniform(30,100,Nneurons)*ms
G.Vreset = uniform(-60,-45,Nneurons)*mV
G.Ibias = 'randn()*5*pA'

# G.a = -0.5*nS
# G.b = 7*pA
# G.tau_m = 5*ms
# G.tau_w = 100*ms
# G.Vreset = -46*mV

In [ ]:
input_current = inpf.get_step_current(1000, 2000, 1*ms, 1.0*pA,Nneurons=Nneurons)

In [ ]:
store()

In [ ]:
restore()
G.stim_amp = 100
%time run(5*second)

In [ ]:
figure(figsize=(16,6))
plot(S.t/ms, S.i, 'k,',lw=0.5)
xlabel('time, ms')
ylabel('neuron #')
title('Raster plot for the neuron group during step current')

In [ ]:
k = randint(Nneurons)
plot(M.t, beautify_spikes(M,S,k)/mV)
xlabel('time, s')
ylabel('Vm, mV')
title('membrane voltage of a randomly chosen neuron')

In [ ]:
array([1/mean(diff((S.t[S.i==k])))])

In [ ]:
def validate_neurons(statemon,spikes,
                     min_spiking_rate = 5*Hz,
                     max_spiking_rate = 300*Hz,
                     reject_restless_prob=0.5):
    N = len(statemon.v)
    invalid = any(abs(statemon.v[:,:])>200*mV,axis=1) + isnan(statemon.v.sum(1))
    late_spikes = array([any(spikes.t[spikes.i==i]>3*second) for i in range(N)])
    
    rates = array([1/mean(diff(spikes.t[spikes.i==i]))/second if len(spikes.t[spikes.i==i])>2 else 0*Hz 
                   for i in range(N)])/second
    invalid += (rates < min_spiking_rate)
    invalid += (rates > max_spiking_rate)
    invalid += (late_spikes)&(rand(N)<reject_restless_prob)
    return ~invalid
    

In [ ]:
%time valid = validate_neurons(M,S)
print(sum(valid)/Nneurons)

In [ ]:
k = where(valid)[0][randint(sum(valid))]

In [ ]:
k,valid[k]

In [ ]:
figure()
plot(M.t/ms,beautify_spikes(M,S,k)/mV)
xlabel('time , ms')
ylabel('Vm, mV')

In [ ]:
def is_good_interneuron(statemon,spikes,k,min_rate=50*Hz,min_omega=5*Hz):
    rate = 1/mean(diff(spikes.t[spikes.i==k])) if len(spikes.t[spikes.i==k])>2 else 0*Hz
    aR = G.a[k]*Rin
    tautau = G.tau_m[k]/G.tau_w[k]
    omega = 4*(aR - 2*(1-tautau**2)/tautau)/G.tau_w[k]
    return(rate>min_rate) and (omega>0.5*min_rate)

def is_good_pyramidal(statemon,spikes,k,max_rate=25*Hz):
    rate = 1/mean(diff(spikes.t[spikes.i==k])) if len(spikes.t[spikes.i==k])>2 else 0*Hz
    aR = G.a[k]*Rin
    tautau = G.tau_m[k]/G.tau_w[k]
    omega = 4*(aR - 2*(1-tautau**2)/tautau)/G.tau_w[k]
    return(rate<=max_rate) and (omega <= 0)

In [ ]:
%time interneurons = array([is_good_interneuron(M,S,k) for k in range(Nneurons)])

In [ ]:
%time pyramidals = array([is_good_pyramidal(M,S,k) for k in range(Nneurons)])

In [ ]:
acc = zeros((Nneurons,2))
for k in range(Nneurons):
    if valid[k]:
        aR = G.a[k]*Rin
        tautau = G.tau_m[k]/G.tau_w[k]
        acc[k] = tautau,aR


In [ ]:
plot(acc[interneurons][:,0],acc[interneurons][:,1],'b.')
plot(acc[pyramidals][:,0],acc[pyramidals][:,1],'r.',alpha=0.1)
xlabel('$\\tau_m/\\tau_w$')
ylabel('aR')

In [ ]:
sum(interneurons)/Nneurons, sum(pyramidals)/Nneurons

In [ ]:
interneuron_pars = dict(a=G.a[interneurons], b=G.b[interneurons],
                        tau_w=G.tau_w[interneurons],tau_m=G.tau_m[interneurons],
                        Ibias=G.Ibias[interneurons],
                        Vreset = G.Vreset[interneurons])
pyramidal_pars = dict(a=G.a[pyramidals], b=G.b[pyramidals],
                        tau_w=G.tau_w[pyramidals],tau_m=G.tau_m[pyramidals],
                        Ibias = G.Ibias[pyramidals],
                        Vreset = G.Vreset[pyramidals])

In [ ]:
len(interneuron_pars['a'])

In [ ]:
def sample_from_class(parameters,sigma=0.01):
    N = len(parameters['a'])
    k = randint(N)
    multipliers = 1 + sigma*randn(10)
    return dict(a = parameters['a'][k]*multipliers[0],
                b = parameters['b'][k]*multipliers[1],
                tau_m = parameters['tau_m'][k]*multipliers[2],
                tau_w = parameters['tau_w'][k]*multipliers[3],
                Vreset = parameters['Vreset'][k]*multipliers[4],
                Ibias = parameters['Ibias'][k]*multipliers[5]
               )


In [ ]:
def update_neuron_parameters(neurons, k, pars):
    neurons.a[k] = pars['a']
    neurons.b[k] = pars['b']
    neurons.tau_m[k] = pars['tau_m']
    neurons.tau_w[k] = pars['tau_w']
    neurons.Vreset[k] = pars['Vreset']
    neurons.Ibias[k] = pars['Ibias']

In [ ]:
def raster_spikes(spikemon,Nexc=None,ax=None,colors='rb'):
    if ax is None:
        f,ax = subplots(1,1)
    if Nexc is None:
        Nexc = max(spikemon.i)
    exc_spikes = spikemon.i<Nexc
    if any(exc_spikes):
        ax.plot(spikemon.t[exc_spikes]/ms, spikemon.i[exc_spikes],',',c=colors[0])
    if any(~exc_spikes):
        ax.plot(spikemon.t[~exc_spikes]/ms, spikemon.i[~exc_spikes],',',c=colors[1])

Here we create a source of "natural" drive to our future network in form of Poisson spike trains from 100 neurons

In [ ]:
start_scope()
P = PoissonGroup(100,  'rand()*25*Hz')
Sp = SpikeMonitor(P)

In [ ]:
run(5*second)

In [ ]:
raster_spikes(Sp,100,colors='gg')

## Now create a new network with 80% pyramidal cells and 20% interneurons

Below we will create a model network with many principal (excitatory) neurons and a smaller amount of interneurons (inhibitory). A random subset of principal neurons will receive "sensory" input in form of Poisson spike trains.

Neurons within the network will be connected with synapses havin both short-time plasticity (Tsodyks & Markram model) and spike-timing dependent plasticity (STDP).

In [ ]:
seed(4022)

In [ ]:
start_scope()


Ninh = 1000
Nexc = 4*Ninh


Nneurons = Nexc + Ninh

defaultclock.dt = 0.1*ms

G = NeuronGroup(Nneurons, AdEx_equations+synaptic_equations,threshold='v>Vcut', reset='v=Vreset; w += b',
                method='exponential_euler')

G.set_states(dict(v=El,w=0))

M = StateMonitor(G, ['v','w','g_e','g_i'], record=True)
S = SpikeMonitor(G,)

In [ ]:
for k in range(Nneurons):
    exemplar_parameters = pyramidal_pars if k < Nexc else interneuron_pars
    #exemplar_parameters = pyramidal_pars# if k < Nexc else interneuron_pars
    sigma = 0.05 if k < Nexc else 0.01
    p = sample_from_class(exemplar_parameters,sigma)
    update_neuron_parameters(G,k,adapting_pars)

In [ ]:
P = PoissonGroup(100,  'rand()*25*Hz')
Sp = SpikeMonitor(P)

### Setting up synaptic connections

In [ ]:
tau_ps = 0.8*second   # facilitation timescale
tau_ns = 1.5*second   # replenishing timescale
tau_stdp = 20*ms     # STDP time constant

p_s0 = 0.6            # ground-state probability of release
epsilon = 0.05        # sparsity synaptic connectivity

Apre0 = 0.01
Apost0 = -Apre0*1.05

w_e = 20*0.05*nS
w_i = 20*1*nS

stdp_value = 0 # change between 0 and 1

tm_plasticity_model = Equations('''
dp_s/dt = (p_s0-p_s)/tau_ps : 1 (event-driven)    # release probability
dn_s/dt = (1-n_s)/tau_ns   : 1    (event-driven)    # fraction of resources available
''')

stdp_model=Equations('''
dApre/dt = -Apre/tau_stdp : 1 (event-driven)    # STDP
dApost/dt = -Apost/tau_stdp : 1(event-driven)   # STDP
w_syn: 1
''')

plasticity_action_pre='''
p_s += p_s0*(1-p_s) # facilitation
r_s = p_s*n_s       # probability of release
n_s -= r_s          # depletion
'''

stdp_action_pre='''
Apre += Apre0
'''

stdp_action_post='''
Apost += Apost0
'''


pre_actions_e = '''
w_syn = clip(w_syn+Apost, (1-stdp_value)*w_e/nS, w_e/nS) 
g_e_post += w_syn*r_s*nS
'''
pre_actions_i = '''
w_syn = clip(w_syn+Apost, (1-stdp_value)*w_i/nS, w_i/nS) 
g_i_post += w_syn*r_s*nS
'''

post_actions_e='''
w_syn = clip(w_syn+Apre,(1-stdp_value)*w_e/nS,w_e/nS)
'''

post_actions_i='''
w_syn = clip(w_syn+Apre,(1-stdp_value)*w_i/nS,w_i/nS)
'''

In [ ]:
S_exc = Synapses(G[:Nexc],G, model=tm_plasticity_model+stdp_model,
                 on_pre=plasticity_action_pre+stdp_action_pre+pre_actions_e,
                 on_post=stdp_action_post+post_actions_e)

                
S_inh = Synapses(G[Nexc:],G, model=tm_plasticity_model+stdp_model,
                 on_pre=plasticity_action_pre+stdp_action_pre+pre_actions_i,
                 on_post=stdp_action_post+post_actions_i)


S_input = Synapses(P,G[:Nexc],model=tm_plasticity_model,
                 on_pre=plasticity_action_pre+'g_e_post += 10*w_e*r_s')


In [ ]:
%%time 

S_input.connect(p=0.1)

S_exc.connect(p=epsilon)
S_exc.delay = '10*ms + 0.1*randn()*ms'

S_inh.connect(p=2*epsilon)
S_inh.delay = '10*ms + 0.1*randn()*ms'

for s in (S_exc,S_inh):
    s.p_s = p_s0
    s.n_s = 1.0

In [ ]:
S_input.active=True
S_exc.active=False
S_inh.active=False

In [ ]:
net = Network(G,P,M,S,Sp,S_exc,S_inh,S_input)

In [ ]:
net.store()

In [ ]:
input_current = inpf.get_step_current(1000, 6000, 1*ms, 1.0*pA,Nneurons=Nneurons)

In [ ]:
net.restore()
S_input.active=True
S_exc.active=False
S_inh.active=False
G.stim_amp = 25

In [ ]:
%time net.run(10*second,report='text')

In [ ]:
%time valid = validate_neurons(M,S)
print(sum(valid)/Nneurons)

In [ ]:
f,axs = subplots(2,1,sharex=True,figsize=(16,8),gridspec_kw=dict(height_ratios=(5,1)))
raster_spikes(S,Nexc,axs[0])
axs[0].set_ylim(0,Nneurons)
raster_spikes(Sp,100,axs[1],colors='gg')
xlim(M.t[0]/ms,M.t[-1]/ms)

In [ ]:
k = randint(Nexc)
plot(M.t/ms, beautify_spikes(M,S,k))

In [ ]:
k = randint(Nexc)
plot(M.t/ms, M.g_e[k]/nS)

### Turn on the synapses

In [ ]:
net.restore()
stdp_value = 0.5 # change between 0 and 1

In [ ]:
S_exc.active=True
S_inh.active=True

In [ ]:
G.stim_amp = 25

%time net.run(10*second,report='text')

In [ ]:
f,axs = subplots(2,1,sharex=True,figsize=(16,8),gridspec_kw=dict(height_ratios=(5,1)))

raster_spikes(S,Nexc,axs[0])
raster_spikes(Sp,100,axs[1],colors='gg')

axs[0].set_ylim(0,Nneurons)

xlim(M.t[0]/ms,M.t[-1]/ms)
axs[0].set_ylabel('# neuron')
xlabel('time')
ylabel('# Poisson generator')

In [ ]:
k = randint(Ninh)+Nexc
print(k)
plot(M.t/ms, beautify_spikes(M,S,k)/mV)

In [ ]:
plot(M.t/ms, M.g_e[k]/nS)